In [58]:
# This is a sample code for the algorithm introduced in the paper
# 'Rollout-Based Charging Strategy for Electric Trucks with Hours-of-Service Regulations'
# Copyright @ Yuchao Li, Ting Bai
# Please cite the paper if the codes are used for academic publication

# The solver used in the sample code for the linear programs is Gurobi
# To check the available solvers in your computer, type 'print(cp.installed_solvers())'  
# Further testing data, and variations of implementaion can be obtained upon request by writing to
# tingbai@kth.se, number127578@gmail.com

In [59]:
import numpy as np
import cvxpy as cp
import itertools
import time
from concurrent.futures import ProcessPoolExecutor, as_completed

In [60]:
# Parameters related to the road model
class road_charge:
    def __init__(self, tau, detour, Pcharging):
        self.tau = tau              # Vector $[\tau_0,\dots,\tau_N]$
        self.detour = detour        # Vector $[d_0,\dots,d_{N-1}]$
        self.Pcharging = Pcharging  # Vector $[P_0,\dots,P_{N-1}]$

In [61]:
# Parameters related to the regulations and market
class regulation_market:
    def __init__(self, Tr, Td, Tbard, DeltaT, price):
        self.Tr = Tr          # Minimum rest time, $T_r$
        self.Td = Td          # Maximum continuous driving time, $T_d$
        self.Tbard = Tbard    # Maximum total travel time per day, $\bar{T}_d$
        self.DeltaT = DeltaT  # Maximum deviation time related to the deadline, $\Delta_T$
        self.price = price    # Electricity price, $\xi_k$ constant for all $k$

In [62]:
class EV_charge:
    
    def __init__(self, Pmax, eusable, eini, Pbar, tp):
        self.Pmax = Pmax      # Maximum charging power of the truck, $P_{\max}$
        self.eusable = eusable# Usable energy of the truck with fully charged battery, $e_f-e_s$
        self.e0 = eini        # Initial energy of the truck, $e_{ini}$
        self.Pbar = Pbar      # Energy consumption per unit time, $\bar{P}$
        self.tp = tp          # Charging preparation time, $p_k$ constant for all $k$
    #------------------------------------------------------------------------------
    # Specify the charging problem given the parameters of the truck, road, and related reguations
    def charge_problem(self,road_charge):
        self.N = road_charge.detour.size
        self.b_charging = np.zeros(self.N)          # Vector $[b_0,\dots,b_{N-1}]$         
        self.b_resting = np.zeros(self.N)           # Vector $[\tilde{b}_0,\dots,\tilde{b}_{N-1}]$
        # Vector $[\bar{P}d_0,\dots,\bar{P}d_{N-1},0]$.
        # The last element is one since the destination has reached.
        self.esafe = np.append(self.Pbar*road_charge.detour.copy(),[0.]) #e_s
        # Vector, elements are $\min{P_k,P_{\max}}$
        self.Pactual = np.minimum(road_charge.Pcharging, self.Pmax)  
    #------------------------------------------------------------------------------
    # Timid base policy, setting $b_k=\tilde{b}_k=1$ for all stations
    def timid_policy(self):
        self.timid_charging = np.ones(self.N)
        self.timid_resting = np.ones(self.N)
    #------------------------------------------------------------------------------
    # Greedy base policy, setting $b_k=\tilde{b}_k=1$ if the truck cannot reach station $k+1$
    def greedy_policy(self,road_charge):
        b = np.zeros(self.N)
        e_current = self.e0
        tau = road_charge.tau
        detour = road_charge.detour

        for i in range(self.N):
            if (e_current-self.Pbar*(tau[i]+tau[i+1]))<self.esafe[i+1]:
                b[i]=1
                e_current = self.eusable-detour[i]*self.Pbar
            else:
                e_current -=self.Pbar*tau[i]
  
        self.greedy_charging = b.copy()
        self.greedy_resting = b.copy()
    #------------------------------------------------------------------------------
    # Computing lower bound
    def lower_bound(self, road_charge, regulation_market):
        N = self.N
        tau = road_charge.tau.copy()
        detour = road_charge.detour.copy()

        Pactual = self.Pactual
        eusable = self.eusable
        esafe = self.esafe.copy()
        e0 = self.e0
        Pbar = self.Pbar
        tp = self.tp
        Tr = regulation_market.Tr
        Td = regulation_market.Td
        Tbard = regulation_market.Tbard
        DeltaT = regulation_market.DeltaT
        Tbound = 1000000
        price = regulation_market.price
        epsilon = 0.001

    
        # if detour.dot(bbar) <= (Tbard-tau.sum())/2:
        # This is the case where the total travel time constraint is fulfilled
        t_charging = cp.Variable((N))                 # Vector, $[t_0,\dots,t_{N-1}]$
        t_extra = cp.Variable((N))                    # Vector, t_extra[i]+t_charging[i]>=Tr
        c_continuous_driving = cp.Variable((N+1))     # Vector, $[c_0,\dots,c_N]$
        e_battery = cp.Variable((N+1))                # Vector, $[e_0,\dots,e_N]$

        delta_T = cp.Variable((N+1))

        b_charging = cp.Variable((N))
        b_resting = cp.Variable((N))
        bbar = cp.Variable((N))
        bhat = cp.Variable((N))
        Delta_ehat = cp.Variable((N))
        chat = cp.Variable((N))
        t_ch_re = cp.Variable((N))


        cost = 0
        constr = []
        constr += [c_continuous_driving[0] == tau[0]]
        constr += [e_battery[0] == e0-Pbar*tau[0]]
        constr += [delta_T[0] == 0]
        for i in range(N):
            constr += [b_charging[i]<=1,b_charging[i]>=0,b_resting[i]<=1,b_resting[i]>=0]
            constr += [bhat[i]<=1,bhat[i]>=0,bbar[i]<=1,bbar[i]>=0]
            constr += [bhat[i]<=b_charging[i],bhat[i]<=b_resting[i],bhat[i]>=b_charging[i]+b_resting[i]-1]
            constr += [bbar[i]>=b_charging[i],bbar[i]>=b_resting[i],bbar[i]<=b_charging[i]+b_resting[i]]
            constr += [Delta_ehat[i]<=b_charging[i]*Tbound,Delta_ehat[i]>=0]
            constr += [t_charging[i]*Pactual[i]-Delta_ehat[i]<=(1-b_charging[i])*Tbound]
            constr += [t_charging[i]*Pactual[i]-Delta_ehat[i]>=0]

            constr += [chat[i]<=b_resting[i]*Tbound,chat[i]>=0]
            constr += [c_continuous_driving[i]-chat[i]<=(1-b_resting[i])*Tbound]
            constr += [c_continuous_driving[i]-chat[i]>=0]

            constr += [t_ch_re[i]<=bhat[i]*Tbound,t_ch_re[i]>=0]
            constr += [t_charging[i]-t_ch_re[i]<=(1-bhat[i])*Tbound]
            constr += [t_charging[i]-t_ch_re[i]>=0]

            constr += [c_continuous_driving[i]<= (Td-detour[i])]
            constr += [c_continuous_driving[i+1] == tau[i+1]+bbar[i]*detour[i]+\
                 (c_continuous_driving[i]+bbar[i]*detour[i])-chat[i]-b_resting[i]*detour[i]]
            constr += [e_battery[i] >= esafe[i]]
            constr += [t_charging[i] <= (eusable-(e_battery[i]-Pbar*tau[i]))/Pactual[i]]
            constr += [e_battery[i+1] == e_battery[i]+Delta_ehat[i]-2*bbar[i]*Pbar*detour[i]-Pbar*tau[i+1]]
            constr += [t_extra[i]+Delta_ehat[i]/Pactual[i]+b_charging[i]*tp >= b_resting[i]*Tr]
            constr += [Delta_ehat[i]/Pactual[i]+b_charging[i]*tp-t_ch_re[i]-bhat[i]*tp <= Tr-epsilon]
            
                

            constr += [t_extra[i] >= 0, t_charging[i] >= 0]
            constr += [t_extra[i] <= b_resting[i]*Tbound, t_charging[i] <= b_charging[i]*Tbound]
            constr += [delta_T[i+1]==delta_T[i]+t_extra[i]+Delta_ehat[i]/Pactual[i]+b_charging[i]*tp]
            cost += t_extra[i]+Delta_ehat[i]/Pactual[i]+b_charging[i]*tp+bbar[i]*2*detour[i]+Delta_ehat[i]/Pactual[i]*price

        constr += [c_continuous_driving[N] <= Td, e_battery[N] >= esafe[N]]
        constr += [delta_T[N] <= DeltaT, detour.T@bbar <= (Tbard-tau.sum())/2]


        problem = cp.Problem(cp.Minimize(cost), constr)
        lower_bd = problem.solve(solver='GUROBI')
        lower_bd_t_charge = np.array(t_charging.value)
        lower_bd_t_extra = np.array(t_extra.value)
        lower_bd_b_charging = np.array(b_charging.value)
        lower_bd_b_resting = np.array(b_resting.value)
        return lower_bd, lower_bd_t_charge, lower_bd_t_extra, lower_bd_b_charging, lower_bd_b_resting
    #------------------------------------------------------------------------------
    # Compute relaxed base policy
    def relaxed_policy(self, road_charge, regulation_market):
        self.relaxed_ld, _, _, charging, resting = self.lower_bound(road_charge, regulation_market)
        charging_tem = (charging>0)
        self.relaxed_charging=charging_tem.astype(float)
        resting_tem = (resting>0)
        self.relaxed_resting=resting_tem.astype(float)
    #------------------------------------------------------------------------------
    # Compute timid, greedy, and relaxed base policies
    def base_policies(self,road_charge,regulation_market):
        self.charge_problem(road_charge)
        self.timid_policy()
        self.greedy_policy(road_charge)
        self.relaxed_policy(road_charge, regulation_market)
    #------------------------------------------------------------------------------
    # Given a set of b_charging and b_resting, compute the optimal times stayed at each station
    def time_schedule(self, b_charging, b_resting, road_charge, regulation_market):
        N = self.N
        tau = road_charge.tau.copy()
        detour = road_charge.detour.copy()

        Pactual = self.Pactual
        eusable = self.eusable
        esafe = self.esafe.copy()
        e0 = self.e0
        Pbar = self.Pbar
        tp = self.tp
        Tr = regulation_market.Tr
        Td = regulation_market.Td
        Tbard = regulation_market.Tbard
        DeltaT = regulation_market.DeltaT
        Tbound = 1000
        price = regulation_market.price
        epsilon = 0.001

        # bbar is the vector of $[\bar{b}_0,\dots,\bar{b}_{N-1}]$
        bbar = np.logical_or(b_charging,b_resting).astype(float)

        if detour.dot(bbar) <= (Tbard-tau.sum())/2:
            
            # This is the case where the total travel time constraint is fulfilled
            t_charging = cp.Variable((N))                 # Vector, $[t_0,\dots,t_{N-1}]$
            delta_e = cp.Variable((N))                    # Vector, $[\Delta e_0,\dots,\Delta e_{N-1}]$
            t_extra = cp.Variable((N))                    # Vector, t_extra[i]+t_charging[i]>=Tr
            c_continuous_driving = cp.Variable((N+1))     # Vector, $[c_0,\dots,c_N]$
            e_battery = cp.Variable((N+1))                # Vector, $[e_0,\dots,e_N]$

            delta_T = cp.Variable((N+1))

            cost = 0
            constr = []
            constr += [c_continuous_driving[0] == tau[0]]
            constr += [e_battery[0] == e0-Pbar*tau[0]]
            constr += [delta_T[0] == 0]
            for i in range(N):
                constr += [c_continuous_driving[i]<= (Td-detour[i])]
                constr += [c_continuous_driving[i+1] == tau[i+1]+bbar[i]*detour[i]+(1-b_resting[i])*(c_continuous_driving[i]+bbar[i]*detour[i])]
                constr += [e_battery[i] >= esafe[i]]
                constr += [delta_e[i]+e_battery[i] <= eusable+Pbar*tau[i]]
                constr += [delta_e[i] == t_charging[i]*Pactual[i]]
                constr += [e_battery[i+1] == e_battery[i]+b_charging[i]*delta_e[i]-2*bbar[i]*Pbar*detour[i]-Pbar*tau[i+1]]
                constr += [t_extra[i]+b_charging[i]*(t_charging[i]+tp) >= b_resting[i]*Tr]
                constr += [(1-b_resting[i])*b_charging[i]*(t_charging[i]+tp) <= Tr-epsilon]
                constr += [t_extra[i] >= 0, t_charging[i] >= 0]
                constr += [t_extra[i] <= b_resting[i]*Tbound, t_charging[i] <= b_charging[i]*Tbound]
                constr += [delta_T[i+1]==delta_T[i]+t_extra[i]+b_charging[i]*(t_charging[i]+tp)]
                cost += t_extra[i]+b_charging[i]*(t_charging[i]+tp)+bbar[i]*2*detour[i]+b_charging[i]*t_charging[i]*price

            constr += [c_continuous_driving[N] <= Td, e_battery[N] >= esafe[N]]
            constr += [delta_T[N] <= DeltaT]

            problem = cp.Problem(cp.Minimize(cost), constr)
            op_value = problem.solve(solver='GUROBI')
            op_t_charge = np.array(t_charging.value)
            op_t_extra = np.array(t_extra.value)
        else:
            op_value = np.inf
            op_t_charge = np.nan*np.ones(N)
            op_t_extra = np.nan*np.ones(N)
            
        return op_value, op_t_charge, op_t_extra # upper bound
    #------------------------------------------------------------------------------
    # Rollout algorithm for solving the charging problem
    def rollout(self, b_charging, b_resting, road_charge, regulation_market):
        rollout = np.inf
        temp = np.inf

        rollout_charging = b_charging.copy()
        rollout_resting = b_resting.copy()
        rollout_t_charge = cp.Variable(self.N) 
        rollout_t_extra = cp.Variable(self.N) 
    
        for i in reversed(range(self.N)):
            b_charging = rollout_charging.copy()
            b_resting = rollout_resting.copy()
            for j in range(4):
                if j == 0:
                    b_charging[i] = 0
                    b_resting[i] = 0
                elif j == 1:
                    b_charging[i] = 0
                    b_resting[i] = 1
                elif j == 2:
                    b_charging[i] = 1
                    b_resting[i] = 0
                else:
                    b_charging[i] = 1
                    b_resting[i] = 1
                temp, op_t_charge, op_t_extra = self.time_schedule(b_charging, b_resting, road_charge, regulation_market)
                if temp is not None:
                    if rollout > temp:
                        rollout_charging = b_charging.copy()
                        rollout_resting = b_resting.copy()
                        rollout = temp
                        rollout_t_charge = op_t_charge
                        rollout_t_extra = op_t_extra
                
    
        return rollout, rollout_t_charge, rollout_t_extra, rollout_charging, rollout_resting

    #------------------------------------------------------------------------------
    # Parametrized implementation of the rollout algorithm
    def rollout_para(self, b_charging_ini, b_resting_ini, road_charge, regulation_market):
        rollout = np.inf
        temp = np.inf

        rollout_charging = b_charging_ini.copy()
        rollout_resting = b_resting_ini.copy()
        rollout_t_charge = rollout_charging.copy()
        rollout_t_extra=rollout_charging.copy()
        
        N = self.N
        tau = road_charge.tau.copy()
        detour = road_charge.detour.copy()

        Pactual = self.Pactual
        eusable = self.eusable
        esafe = self.esafe.copy()
        e0 = self.e0
        Pbar = self.Pbar
        tp = self.tp
        Tr = regulation_market.Tr
        Td = regulation_market.Td
        Tbard = regulation_market.Tbard
        DeltaT = regulation_market.DeltaT
        Tbound = 1000
        price = regulation_market.price
        epsilon = 0.001
        
        b_charging = cp.Parameter(N,nonneg=True)
        b_resting = cp.Parameter(N,nonneg=True)
        bbar = cp.Parameter(N,nonneg=True)
        bhat = cp.Parameter(N,nonneg=True)
            
            
        t_charging = cp.Variable((N))                 # Vector, $[t_0,\dots,t_{N-1}]$
        delta_e = cp.Variable((N))                    # Vector, $[\Delta e_0,\dots,\Delta e_{N-1}]$
        t_extra = cp.Variable((N))                    # Vector, t_extra[i]+t_charging[i]>=Tr
        c_continuous_driving = cp.Variable((N+1))     # Vector, $[c_0,\dots,c_N]$
        e_battery = cp.Variable((N+1))                # Vector, $[e_0,\dots,e_N]$

        delta_T = cp.Variable((N+1))

        cost = 0
        constr = []
        constr += [c_continuous_driving[0] == tau[0]]
        constr += [e_battery[0] == e0-Pbar*tau[0]]
        constr += [delta_T[0] == 0]
        for i in range(N):
            constr += [c_continuous_driving[i]<= (Td-detour[i])]
            constr += [c_continuous_driving[i+1] == tau[i+1]+bbar[i]*detour[i]+\
                      (1-b_resting[i])*c_continuous_driving[i]+bbar[i]*detour[i]-b_resting[i]*detour[i]]
            constr += [e_battery[i] >= esafe[i]]
            constr += [delta_e[i]+e_battery[i] <= eusable+Pbar*tau[i]]
            constr += [delta_e[i] == t_charging[i]*Pactual[i]]
            constr += [e_battery[i+1] == e_battery[i]+b_charging[i]*delta_e[i]-2*bbar[i]*Pbar*detour[i]-Pbar*tau[i+1]]
            constr += [t_extra[i]+b_charging[i]*(t_charging[i]+tp) >= b_resting[i]*Tr]
            constr += [(1-b_resting[i])*b_charging[i]*(t_charging[i]+tp) <= Tr-epsilon]
            constr += [t_extra[i] >= 0, t_charging[i] >= 0]
            constr += [t_extra[i] <= b_resting[i]*Tbound, t_charging[i] <= b_charging[i]*Tbound]
            constr += [delta_T[i+1]==delta_T[i]+t_extra[i]+b_charging[i]*(t_charging[i]+tp)]
            cost += t_extra[i]+b_charging[i]*(t_charging[i]+tp)+bbar[i]*2*detour[i]+b_charging[i]*t_charging[i]*price

        constr += [c_continuous_driving[N] <= Td, e_battery[N] >= esafe[N]]
        constr += [delta_T[N] <= DeltaT, detour.T@bbar <= (Tbard-tau.sum())/2]
        
        prob = cp.Problem(cp.Minimize(cost), constr)
        
        comb = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

         
        for i in range(self.N):
            
            for j in range(4):
                b_charging_tem = rollout_charging.copy()
                b_resting_tem = rollout_resting.copy()

                b_charging_tem[i] = comb[j, 0]
                b_resting_tem[i] = comb[j, 1]

                b_charging.value = b_charging_tem.copy()
                b_resting.value = b_resting_tem.copy()
                bbar.value = np.logical_or(b_charging_tem.copy(), b_resting_tem.copy()).astype(float)
                bhat.value = np.multiply(b_charging_tem, b_resting_tem)

                temp = prob.solve(solver='GUROBI')
                if temp is not None:
                    if rollout > temp:
                        rollout_charging = b_charging_tem.copy()
                        rollout_resting = b_resting_tem.copy()
                        rollout = temp
                        rollout_t_charge = np.array(t_charging.value)
                        rollout_t_extra = np.array(t_extra.value)
                
        
        return rollout, rollout_charging, rollout_resting, rollout_t_charge, rollout_t_extra
    
    # Rollout for earch combination
    def rollout_per_combination(self, i, b_charging, b_resting, road_charge, regulation_market):
        temp = np.inf
        for j in range(4):
            if j == 0:
                b_charging[i] = 0
                b_resting[i] = 0
            elif j == 1:
                b_charging[i] = 0
                b_resting[i] = 1
            elif j == 2:
                b_charging[i] = 1
                b_resting[i] = 0
            else:
                b_charging[i] = 1
                b_resting[i] = 1
            temp, op_t_charge, op_t_extra = self.time_schedule(b_charging, b_resting, road_charge, regulation_market)
            if temp < np.inf:
                return temp, b_charging.copy(), b_resting.copy(), op_t_charge, op_t_extra
        return np.inf, b_charging.copy(), b_resting.copy(), None, None
    
    # Parallel implementation of rollout
    def rollout_parallel(self, b_charging, b_resting, road_charge, regulation_market):
        rollout = np.inf
        rollout_charging = b_charging.copy()
        rollout_resting = b_resting.copy()
        rollout_t_charge = b_charging.copy()
        rollout_t_extra = b_charging.copy()

        with ProcessPoolExecutor() as executor:
            futures = {executor.submit(self.rollout_per_combination, i, b_charging.copy(), b_resting.copy(), road_charge, regulation_market) 
                       for i in reversed(range(self.N))}
            for future in as_completed(futures):
                temp, temp_charging, temp_resting, temp_t_charge, temp_t_extra = future.result()
                if temp < rollout:
                    rollout = temp
                    rollout_charging = temp_charging
                    rollout_resting = temp_resting
                    rollout_t_charge = temp_t_charge
                    rollout_t_extra = temp_t_extra

        return rollout, rollout_t_charge, rollout_t_extra, rollout_charging, rollout_resting
        
    
    #------------------------------------------------------------------------------
    # Computing the best charging solution by enumerating all the combinations of the binary variables
    def best_exhaust(self, road_charge, regulation_market):
        N = self.N
        best=np.inf
        current=np.inf
        best_charging=np.zeros(N)
        best_resting=np.zeros(N)

        vector=[0,1]
        b_charge_combinations=list(itertools.product(vector,repeat=N))# all the combinations of the binary variables for b_charging
        b_rest_combinations=list(itertools.product(vector,repeat=N))
        for b_1 in b_charge_combinations:
            for b_2 in b_rest_combinations:
                current,t_charge,t_extra=self.time_schedule(np.array(b_1), np.array(b_2),road_charge, regulation_market)
                if current is not None:
                    if best > current:
                        best_charging = np.array(b_1).copy()
                        best_resting = np.array(b_2).copy()
                        best_t_charge=t_charge.copy()
                        best_t_extra=t_extra.copy()
                        best = current.copy()
                 

        self.best_charging = best_charging.copy()
        self.best_resting = best_resting.copy()
        self.best_t_charge = best_t_charge.copy()
        self.best_t_extra = best_t_extra.copy()
        self.best = best


In [ ]:
# Testing data used in the paper
#N=5
#{'tau': [23.4219, 68.9947, 24.1869, 107.553, 89.3284, 24.4093],
# 'detour': [11.8715, 13.0527, 10.1167, 15.2695, 17.0627],
# 'DeltaT': 150}
#N=6
#{'tau': [24.7328, 17.3314, 52.2553, 24.4523, 76.4669, 35.5783, 79.6548],
# 'detour': [25.7005, 4.8874, 4.513, 20.3038, 6.0249, 3.2189],
# 'DeltaT': 150}
#N=7
#{'tau': [30.1441, 39.1227, 54.9621, 70.438, 15.7162, 73.1713, 62.0197, 26.0694],
# 'detour': [9.8573, 7.7971, 3.6675, 0.5792, 1.8247, 6.0237, 7.4384],
# 'DeltaT': 150}
#N=8
#{'tau': [36.4262, 70.3561, 15.5605, 73.1098, 78.214, 22.4032, 53.2017, 17.8092, 25.7993],
# 'detour': [2.2147, 2.2827, 1.8254, 6.023, 19.2596, 4.2088, 10.57, 24.568],
# 'DeltaT': 150}
#N=9
#{'tau': [29.6962, 37.8472, 102.0682, 73.6421, 17.1241, 69.6287, 54.3168, 23.1439, 31.5338, 41.805, 43.3317],
# 'detour': [3.4023, 0.7798, 6.0249, 1.8252, 2.3276, 3.6682, 0.1866, 3.7495, 6.1948],
# 'DeltaT': 220}
#N=10
#{'tau': [30.1441, 39.1227, 54.9621, 70.438, 15.7162, 73.1713, 78.1987, 22.4278, 66.7064, 55.2526, 28.2965],
# 'detour': [9.8573, 7.7971, 3.6675, 0.5792, 1.8247, 6.0237, 19.2602, 4.2078, 0.305, 11.0378],
# 'DeltaT': 220}

In [63]:
# Example run for the code

tau_list=[23.4219, 68.9947, 24.1869, 107.553, 89.3284, 24.4093]

detour_list=[11.8715, 13.0527, 10.1167, 15.2695, 17.0627]
# compute the lower bound of the initial battery
tau_0=tau_list[0]
detour_0=detour_list[0]
        
# Parameter setting
tau=np.array(tau_list)
detour=np.array(detour_list)
Pcharging = np.array([300, 300, 300, 300, 300])/60
Pmax = 375/60
Pbar = 110/60
tp = 6
Tr = 45
Td = 4.5*60
Tbard = 9*60
DeltaT = 2.5*60
price = 0.36/0.4
Pbar=110/60
eusable = 468
eini = 0.8*eusable

In [64]:
road = road_charge(tau, detour, Pcharging)
regulation = regulation_market(Tr, Td, Tbard, DeltaT, price)
ev = EV_charge(Pmax, eusable, eini, Pbar, tp)

In [65]:
ev.base_policies(road,regulation)
print(ev.esafe,ev.greedy_charging)
ev.time_schedule(ev.timid_charging,ev.timid_resting,road,regulation)

[21.76441667 23.92995    18.54728333 27.99408333 31.28161667  0.        ] [0. 0. 1. 0. 0.]


(inf, array(None, dtype=object), array(None, dtype=object))

In [66]:
# Test parametrized implementation of rollout
pre = time.perf_counter()
ev.rollout_para(ev.relaxed_charging,ev.relaxed_resting, road, regulation)
print(time.perf_counter()-pre)

1.661758553999789


In [67]:
# Test parallel implementation of rollout
pre = time.perf_counter()
ev.rollout_parallel(ev.relaxed_charging,ev.relaxed_resting, road, regulation)
print(time.perf_counter()-pre)

0.3613530269999501


In [68]:
pre = time.perf_counter()
ev.best_exhaust(road, regulation)
print(time.perf_counter()-pre)

69.24578801500002
